In [1]:
!pip install dash pandas plotly


In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load your dataset
# Assuming 'df' is your pandas DataFrame
# Replace 'your_dataset.csv' with the actual file path or URL
df = pd.read_csv('owid-co2-data.csv')

# Dash App
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1('Combined Emission Dashboard'),

    # Tabs for Cement Industry and Country-wise Emission Overview
    dcc.Tabs([
        dcc.Tab(label='Cement Industry', children=[
            dcc.Graph(
                id='share-of-global-cement-co2',
                figure=px.line(df[['year', 'cumulative_cement_co2']], x='year', y='cumulative_cement_co2',
                               labels={'cumulative_cement_co2': 'Cumulative Cement CO2 Emissions'},
                               title='Cumulative Cement CO2 Emissions Over Time')
            ),

            dcc.Graph(
                id='cement-co2-per-capita',
                figure=px.line(df[['year', 'cement_co2_per_capita', 'cement_co2']], x='year',
                               y=['cement_co2_per_capita', 'cement_co2'],
                               labels={'value': 'Emissions'},
                               title='Cement CO2 Per Capita and Contribution to Overall Emissions')
            )
        ]),

        dcc.Tab(label='Country-wise Overview', children=[
            # Dropdown for selecting specific years
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': year, 'value': year} for year in df['year'].unique()],
                value=df['year'].max(),
                multi=False,
                style={'width': '50%'}
            ),

            # World map with countries color-coded based on their total CO2 emissions
            dcc.Graph(
                id='world-map',
            ),

            # Additional charts for per capita emissions, GDP correlation, and population growth
            dcc.Graph(
                id='per-capita-emissions',
            ),

            dcc.Graph(
                id='gdp-correlation',
            ),

            dcc.Graph(
                id='population-growth',
            ),
        ]),
    ])
])

# Callback for updating the charts based on the selected year
@app.callback(
    [Output('world-map', 'figure'),
     Output('per-capita-emissions', 'figure'),
     Output('gdp-correlation', 'figure'),
     Output('population-growth', 'figure')],
    [Input('year-dropdown', 'value')]
)
def update_charts(selected_year):
    # Filter data for the selected year
    filtered_df = df[df['year'] == selected_year]

    # World map with countries color-coded based on their total CO2 emissions
    map_fig = px.choropleth(filtered_df,
                            locations='iso_code',
                            color='co2',
                            hover_name='country',
                            title=f'Total CO2 Emissions by Country - {selected_year}')

    # Per capita emissions
    per_capita_fig = px.bar(filtered_df,
                            x='country',
                            y='co2_per_capita',
                            title=f'Per Capita CO2 Emissions by Country - {selected_year}')

    # GDP correlation
    gdp_correlation_fig = px.scatter(filtered_df,
                                     x='gdp',
                                     y='co2',
                                     color='country',
                                     title=f'GDP Correlation with CO2 Emissions - {selected_year}')

    # Population growth
    population_growth_fig = px.line(filtered_df,
                                    x='country',
                                    y='population',
                                    title=f'Population Growth by Country - {selected_year}')

    return map_fig, per_capita_fig, gdp_correlation_fig, population_growth_fig


if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
'''ANALYSIS REPORT

Overall, cumulative CO2 emissions from cement production have increased dramatically since 1750. In 2022, cumulative CO2 emissions 
from cement production were about 1.7 trillion tonnes.
The rate of increase in CO2 emissions from cement production has accelerated in recent decades. This is due in part to the rapid growth of 
the construction industry in many developing countries.
China is the largest emitter of CO2 from cement production, followed by India and the United States. In 2022, China emitted about 30% of 
global CO2 emissions from cement production, India emitted about 12%, and the United States emitted about 6%.

In [ ]:
'''The countries with the highest per capita CO2 emissions are the United States, Canada, and Australia. These countries have large economies and 
are heavily reliant on fossil fuels for energy.
The countries with the lowest per capita CO2 emissions are many African countries, including Burundi, Malawi, and Niger. These countries are generally
poor and have small economies.
There is a large variation in per capita CO2 emissions between countries. The United States, for example, has per capita CO2 emissions that are more
than 10 times higher than those of Niger.
There are a number of factors that contribute to the variation in per capita CO2 emissions between countries. These include:

Level of economic development: Countries with higher levels of economic development tend to have higher per capita CO2 emissions. 
This is because economic activity is often associated with energy consumption, and fossil fuels are the most common source of energy.
Reliance on fossil fuels: Countries that are more reliant on fossil fuels for energy tend to have higher per capita CO2 emissions.
Energy efficiency: Countries that are more energy efficient tend to have lower per capita CO2 emissions.
Population density: Countries with higher population densities tend to have lower per capita CO2 emissions. This is because people in 
densely populated countries tend to live in cities, where they have access to public transportation and other energy-efficient forms of transportation.

In [ ]:
'''The charts here can be interacted by selecting the particular year one want to analyse the population growth , GDP Co-relation with CO2 emissions etc. The chart shows the GDP Correlation with CO2 Emissions for various countries in 2000.

The chart currently indicates a weak positive correlation between GDP and CO2 emissions. This means that as GDP increases, CO2 emissions also tend to 
increase, but the relationship is not very strong. There are a number of countries that deviate from this trend, such as:

Norway: Norway has a relatively high GDP but low CO2 emissions. This is because Norway is a major producer of hydroelectricity, which is a clean source
of energy.
Iceland: Iceland also has a high GDP and low CO2 emissions. This is because Iceland gets most of its energy from geothermal power.
Bolivia: Bolivia has a relatively low GDP and low CO2 emissions. This is because Bolivia is a poor country with a small economy.